<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/johnjdailey/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module2-convolutional-neural-networks/data/train/mountain/art1131.jpg?raw=1)|![](https://github.com/johnjdailey/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module2-convolutional-neural-networks/data/validation/forest/cdmc317.jpg?raw=1)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [19]:
# Load a pretrained network from Keras, ResNet50 

import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

In [20]:
# Turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes.

for layer in resnet.layers:
    layer.trainable = False

In [25]:
# Imports

import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

_URL = 'https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module2-convolutional-neural-networks/data.zip?raw=true'

path_to_zip = tf.keras.utils.get_file('./data.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

In [4]:
# Instantiate train and validation directories

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [5]:
# Instantiate train and validation data

train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training mountain pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training forest pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation mountain pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation forest pictures

In [6]:
# Instantiate for number of images for train, validation, and total

num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [7]:
# Print number of images for train, validation, and total

print('total training mountain images:', num_mountain_tr)
print('total training forest images:', num_forest_tr)

print('total validation mountain images:', num_mountain_val)
print('total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training mountain images: 254
total training forest images: 270
total validation mountain images: 125
total validation forest images: 62
--
Total training images: 524
Total validation images: 187


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [8]:
# Instantiate convolutional neural network parameters

batch_size = 16
epochs = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [9]:
# Rescale the Image Data Generator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [10]:
# Instantiate the train image generator

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [11]:
# Instantiate the validation image generator

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


## Instatiate Model

In [12]:
# Instantiate the model and keep only the feature processing portions of our network.

x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [13]:
# Compile the model

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Fit Model

In [14]:
# Fit the model

history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 6s 201ms/step - loss: 0.8392 - accuracy: 0.5629 - val_loss: 0.6963 - val_accuracy: 0.5795
Epoch 2/50
32/32 [==============================] - 5s 154ms/step - loss: 0.6643 - accuracy: 0.6008 - val_loss: 0.5469 - val_accuracy: 0.6875
Epoch 3/50
32/32 [==============================] - 5s 153ms/step - loss: 0.5217 - accuracy: 0.7964 - val_loss: 0.4961 - val_accuracy: 0.6989
Epoch 4/50
32/32 [==============================] - 5s 153ms/step - loss: 0.4885 - accuracy: 0.8084 - val_loss: 0.5369 - val_accuracy: 0.7955
Epoch 5/50
32/32 [==============================] - 5s 156ms/step - loss: 0.4669 - accuracy: 0.7764 - val_loss: 0.6795 - val_accuracy: 0.5682
Epoch 6/50
32/32 [==============================] - 5s 156ms/step - loss: 0.4284 - accuracy: 0.8223 - val_loss: 0.4167 - val_accuracy: 0.8523
Epoch 7/50
32/32 [==============================] - 5s 155ms/step - loss: 0.3734 - accuracy: 0.8603 - val_loss: 0.3764 - val_accuracy: 0.8182
Epoch 

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [27]:
# Define the Model

custom_model = Sequential([
                           Conv2D(32,3, activation="relu", input_shape=(224,224,3)),
                           MaxPool2D(2,2),
                           Conv2D(64,3, activation="relu", input_shape=(224,224,3)),
                           MaxPool2D(2,2),
                           Conv2D(128,3, activation="relu", input_shape=(224,224,3)),
                           MaxPool2D(2,2),
                           Flatten(),
                           Dense(64, activation="relu"),
                           Dropout (0.1),
                           Dense(1, activation="sigmoid")
])

In [28]:
# Compile Model

custom_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [29]:
# Fit Model

history = custom_model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 3s 83ms/step - loss: 0.7902 - accuracy: 0.6806 - val_loss: 0.4107 - val_accuracy: 0.8636
Epoch 2/50
32/32 [==============================] - 2s 73ms/step - loss: 0.2859 - accuracy: 0.8882 - val_loss: 0.2402 - val_accuracy: 0.9318
Epoch 3/50
32/32 [==============================] - 2s 75ms/step - loss: 0.2001 - accuracy: 0.9341 - val_loss: 0.2160 - val_accuracy: 0.9205
Epoch 4/50
32/32 [==============================] - 2s 74ms/step - loss: 0.2414 - accuracy: 0.9162 - val_loss: 0.2116 - val_accuracy: 0.9205
Epoch 5/50
32/32 [==============================] - 2s 74ms/step - loss: 0.2081 - accuracy: 0.9122 - val_loss: 0.2743 - val_accuracy: 0.8807
Epoch 6/50
32/32 [==============================] - 2s 75ms/step - loss: 0.1484 - accuracy: 0.9541 - val_loss: 0.2996 - val_accuracy: 0.8750
Epoch 7/50
32/32 [==============================] - 2s 73ms/step - loss: 0.1228 - accuracy: 0.9561 - val_loss: 0.3769 - val_accuracy: 0.8750
Epoch 8/50
32

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [33]:
# Instantiate the train and validation image generators

train_image_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=None,
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)

val_image_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=None,
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)

train_data_gen = train_image_gen.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = val_image_gen.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 533 images belonging to 2 classes.
Found 195 images belonging to 2 classes.


In [34]:
# Fit Custom Model with Image Manipulations

history = custom_model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 10s 308ms/step - loss: 0.6920 - accuracy: 0.5190 - val_loss: 0.7106 - val_accuracy: 0.3636
Epoch 2/50
32/32 [==============================] - 10s 303ms/step - loss: 0.6856 - accuracy: 0.5309 - val_loss: 0.6894 - val_accuracy: 0.3920
Epoch 3/50
32/32 [==============================] - 10s 300ms/step - loss: 0.6997 - accuracy: 0.5449 - val_loss: 0.7128 - val_accuracy: 0.3693
Epoch 4/50
32/32 [==============================] - 10s 301ms/step - loss: 0.6905 - accuracy: 0.5309 - val_loss: 0.6974 - val_accuracy: 0.3750
Epoch 5/50
32/32 [==============================] - 10s 299ms/step - loss: 0.6681 - accuracy: 0.5808 - val_loss: 0.6662 - val_accuracy: 0.5739
Epoch 6/50
32/32 [==============================] - 10s 305ms/step - loss: 0.6147 - accuracy: 0.6926 - val_loss: 0.4936 - val_accuracy: 0.7727
Epoch 7/50
32/32 [==============================] - 10s 300ms/step - loss: 0.5682 - accuracy: 0.7745 - val_loss: 0.5885 - val_accuracy: 0.6307

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language